In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
import time
finalScore = 0

In [2]:
def arrange_points(points):
    
    top_left = points[np.argmin(points.sum(axis=1))]
    bottom_right = points[np.argmax(points.sum(axis=1))]
    
    
    top_left_diff = top_left - points
    
    idx = 0
    min_val = np.inf
    for i in range(len(top_left_diff)):
        
        if i == top_left or i == bottom_right:
            continue
        
        if top_left_diff[i,0] < min_val:
            min_val = top_left_diff[i,0]
            idx = i
    
    bottom_left = points[idx]
    
    
    
    print(top_left_diffs)

In [3]:
def largestCC(image):
    image = image.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)
    sizes = stats[:, -1]
#     print(nb_components)
    max_label = 0
    max_size = sizes[0]
    for i in range(0, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255
    
#     cv2.imshow("Biggest component", img2)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    return img2

In [18]:
def readSolutionImage(frame):
#     solution_img="sol.jpg"
    solution_img = frame
    score = 0.0
    fig = plt.figure(figsize=(10, 7))
    rows = 4
    columns = 2
    i = 1
#     img = cv2.imread("sol.jpg")
    img = frame
    original = img.copy()
    original_padded = np.zeros((img.shape[0]+50,img.shape[1]+50,img.shape[2])).astype(np.uint8)
    original_padded[25:img.shape[0]+25,25:img.shape[1]+25,:] = original
    #img = cv2.imread("4.jpeg")
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    kernel = np.ones((5,5),np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations= 4)

    #fig.add_subplot(rows, columns, i)
    #plt.imshow(img)

    #         cv2.imshow('a',img)
    #         cv2.waitKey()
    #         cv2.destroyAllWindows()


    low = np.array([0, 0, 0])
    high = np.array([255, 40, 255]) #Calcukate for all values from 15-40, use onw with the most area
    mask = cv2.inRange(hsv, low, high)
    res = cv2.cvtColor(cv2.bitwise_and(hsv, hsv, mask=mask), cv2.COLOR_HSV2BGR)
    padded_img = np.zeros((img.shape[0]+50,img.shape[1]+50,img.shape[2])).astype(np.uint8)
    padded_img[25:img.shape[0]+25,25:img.shape[1]+25,:] = res
    img = padded_img

#     cv2.imshow('a',img)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (20,20,img.shape[1]-20,img.shape[0]-20)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img = img*mask2[:,:,np.newaxis]

    #         cv2.imshow('a',img)
    #         cv2.waitKey()
    #         cv2.destroyAllWindows()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    # Edge Detection.
    canny = cv2.Canny(gray, 0, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Blank canvas.
    con = np.zeros_like(img)
    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Keeping only the largest detected contour.
    page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    #con = cv2.drawContours(original_padded, page[:1], -1, (0, 255, 255), 3)


    # Blank canvas.
    con = np.zeros_like(img)
    # Loop over the contours.
    for c in page:
    # Approximate the contour.
        epsilon = 0.02 * cv2.arcLength(c, True)
        corners = cv2.approxPolyDP(c, epsilon, True)
    # If our approximated contour has four points
        if len(corners) == 4:
            break
    cv2.drawContours(original_padded, c, -1, (0, 255, 255), 3)
    cv2.drawContours(original_padded, corners, -1, (0, 255, 0), 10)
    #Sorting the corners and converting them to desired shape.
    corners = sorted(np.concatenate(corners).tolist())

    # Displaying the corners.
    for index, c in enumerate(corners):
        character = chr(65 + index)
        cv2.putText(original_padded, character, tuple(c), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)

    #print(corners)


    pts = np.array(corners)
    print(corners[0])
    print(corners[1])
    print(corners[2])
    print(corners[3])
    #         arrange_points(pts)

    #cv2.drawContours(img, rect_cnts[:1], -1, (0,255,0), 3)

    #fig.add_subplot(rows, columns, i)
    #plt.imshow(original_padded)


    #cv2.imshow('',original_padded)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()


    i += 1


    #     return score
    temp = corners[1]
    corners[1] = corners[2]
    corners[2] = temp

    pts1 = np.float32([[corners[0][0],corners[0][1]],[corners[1][0],corners[1][1]],[corners[2][0],corners[2][1]],[corners[3][0],corners[3][1]]])
    pts2 = np.float32([[0,0],[img.shape[0],0],[0,img.shape[1]],[img.shape[0],img.shape[1]]])

    M = cv2.getPerspectiveTransform(pts1,pts2)

    dst = cv2.warpPerspective(img,M,(img.shape[0],img.shape[1]))
    dst = cv2.resize(dst,(1000,1200))


#     cv2.imshow('a',dst)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

    smallerArea = dst[int(dst.shape[1]/4):int(dst.shape[1] - dst.shape[1]/20),int(dst.shape[1]/9):int(dst.shape[0] - dst.shape[0]/9)]

#     cv2.imshow('a',smallerArea)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

    mcqs = []
    gray = cv2.cvtColor(smallerArea, cv2.COLOR_BGR2GRAY)
    # gray = cv2.GaussianBlur(gray, (11, 11), 0)
    # Edge Detection.
    canny = cv2.Canny(gray, 50, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 2)))

#     cv2.imshow('a',canny)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    # Blank canvas.
    con = np.zeros_like(smallerArea)
    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    # (contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

    # Plotting the contours
    contoursFrame = smallerArea.copy()
    contoursFrame = cv2.drawContours(contoursFrame, contours, -1, (255, 0, 255), 4)

    # Keeping only the largest detected contour.
    # page = sorted(contours, key=cv2.contourArea, reverse=True)
    #con = cv2.drawContours(original_padded, page[:1], -1, (0, 255, 255), 3)

#     cv2.imshow('a',contoursFrame)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    for i in range(0, len(page)):    #sorting wrt contours
        for j in range(i+1, len(page)):    
            x, y, w, h = cv2.boundingRect(page[i])
            x2, y2, w2, h2 = cv2.boundingRect(page[j])
            if(w*h < w2*h2):    
                temp = page[i];    
                page[i] = page[j];    
                page[j] = temp;           

    count = 0
    xCoords = []
    yCoords = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w > 250 and h >50 and h < 100 and w < 350:# and w < 500:
            mcqs.append(smallerArea[y:y+h,x:x+w])
            xCoords.append(x)
            yCoords.append(y)
    #         cv2.imshow('a',smallerArea[y:y+h,x:x+w])
    #         cv2.waitKey()
    #         cv2.destroyAllWindows()
            count+=1
    print(f'count: {count}')
    
    if(count < 20):
        return 0

    #Sorting MCQs list with respect to y coordinates first then with respect to x coordinates to make them in order
    for i in range(0, len(mcqs)):    #sorting wrt x
        for j in range(i+1, len(mcqs)):    
            if(xCoords[i] > xCoords[j]):    
                temp = mcqs[i]    
                mcqs[i] = mcqs[j]    
                mcqs[j] = temp   
                temp = xCoords[i]
                xCoords[i] = xCoords[j]
                xCoords[j] = temp
                temp = yCoords[i]
                yCoords[i] = yCoords[j]
                yCoords[j] = temp

    for i in range(0, 10):    #sorting wrt y
        for j in range(i+1, 10):    
            if(yCoords[i] > yCoords[j]):    
                temp = mcqs[i];    
                mcqs[i] = mcqs[j]    
                mcqs[j] = temp 
                temp = xCoords[i]
                xCoords[i] = xCoords[j]
                xCoords[j] = temp
                temp = yCoords[i]
                yCoords[i] = yCoords[j]
                yCoords[j] = temp

    for i in range(10, 20):    #sorting wrt y
        for j in range(i+1, 20):    
            if(yCoords[i] > yCoords[j]):    
                temp = mcqs[i]
                mcqs[i] = mcqs[j] 
                mcqs[j] = temp 
                temp = xCoords[i]
                xCoords[i] = xCoords[j]
                xCoords[j] = temp
                temp = yCoords[i]
                yCoords[i] = yCoords[j]
                yCoords[j] = temp

    #splitting mcqs in 5 regions
    mcqsSplitted=[]
    for i in mcqs:
        j = int((i.shape[1]-40)/5)
        currMcq = []
        currMcq.append(i[:,20:20+j])
        currMcq.append(i[:,j+20:20+j*2])
        currMcq.append(i[:,j*2+20:20+j*3])
        currMcq.append(i[:,j*3+20:20+j*4])
        currMcq.append(i[:,j*4+20:20+j*5])
    #     cv2.imshow('a',currMcq[4])
    #     cv2.waitKey()
    #     cv2.destroyAllWindows()
        mcqsSplitted.append(currMcq)
    #     break

    totalCount=0
    answers = []
    for mcq in mcqsSplitted:
        countOfAnswers=0
        for j in range(0,5):
            hsvb = cv2.cvtColor(mcq[j], cv2.COLOR_BGR2HSV)
            black_lower=np.array([0,0,0],np.uint8)
            black_upper=np.array([255,255,80],np.uint8)
            black=cv2.inRange(hsvb,black_lower,black_upper)

            blackc = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)
            extractCC = largestCC(black)
            ROI = 0
            yellowY, yellowX = np.where(extractCC==0)
            if(len(yellowY) == 0 or len(yellowX) == 0):# no black color
                ROI = 0
            else:#black color
                top, bottom = min(yellowY), max(yellowY)
                left, right = min(yellowX), max(yellowX)
    #             print(top,bottom,left,right)
                ROI = mcq[j][top:bottom, left:right]
                noOfBlackPixels = 0
                for i in range(left,right):
                    for k in range(top,bottom):
                        if extractCC[k,i] == 0:
                            noOfBlackPixels+=1
    #             cv2.imshow('a',extractCC[top:bottom, left:right])
    #             cv2.waitKey()
    #             cv2.destroyAllWindows()
                h = abs(top-bottom)
                w = abs(right-left)
                if noOfBlackPixels > 400:
                    countOfAnswers+=1
                    answers.append(j)
        if(countOfAnswers==0): #no answer
            answers.append(-1)
        elif(countOfAnswers>1): #>1 answers
            for x in range(countOfAnswers):
                answers.pop()
            answers.append(-1)
        totalCount+=1

    #     break
    print(f'totalCount:{totalCount}')

    for i in range(0,len(answers)):
        if answers[i] == 0:
            answers[i] = 'A'
        elif answers[i] == 1:
            answers[i] = 'B'
        elif answers[i] == 2:
            answers[i] = 'C'
        elif answers[i] == 3:
            answers[i] = 'D'
        elif answers[i] == 4:
            answers[i] = 'E'
        else:
            answers[i]='I'

    #SCORING
    print(answers)

    # open file in write mode
    with open(r'C:/Users/Umer Mukhtar/Desktop/DIP_Project/sol.txt', 'w') as fp:
        for item in answers:
            # write each item on a new line
            fp.write("%s\n" % item)
    #     print('Done')
    return totalCount


In [ ]:
tC = 0
cap = cv2.VideoCapture('http://192.168.100.11:8080/video')
while(tC!=20):
    ret, frame = cap.read()
    cv2.imshow('frame',frame)
    tC = readSolutionImage(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


In [18]:
def evaluateSolutionImage(frame):
    finalScore = 0
#     solution_img="sol.jpg"
    solution_img = frame
    score = 0.0
    fig = plt.figure(figsize=(10, 7))
    rows = 4
    columns = 2
    i = 1
#     img = cv2.imread("sol.jpg")
    img = frame
    original = img.copy()
    original_padded = np.zeros((img.shape[0]+50,img.shape[1]+50,img.shape[2])).astype(np.uint8)
    original_padded[25:img.shape[0]+25,25:img.shape[1]+25,:] = original
    #img = cv2.imread("4.jpeg")
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    kernel = np.ones((5,5),np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations= 4)

    #fig.add_subplot(rows, columns, i)
    #plt.imshow(img)

    #         cv2.imshow('a',img)
    #         cv2.waitKey()
    #         cv2.destroyAllWindows()


    low = np.array([0, 0, 0])
    high = np.array([255, 40, 255]) #Calcukate for all values from 15-40, use onw with the most area
    mask = cv2.inRange(hsv, low, high)
    res = cv2.cvtColor(cv2.bitwise_and(hsv, hsv, mask=mask), cv2.COLOR_HSV2BGR)
    padded_img = np.zeros((img.shape[0]+50,img.shape[1]+50,img.shape[2])).astype(np.uint8)
    padded_img[25:img.shape[0]+25,25:img.shape[1]+25,:] = res
    img = padded_img

#     cv2.imshow('a',img)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (20,20,img.shape[1]-20,img.shape[0]-20)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img = img*mask2[:,:,np.newaxis]

    #         cv2.imshow('a',img)
    #         cv2.waitKey()
    #         cv2.destroyAllWindows()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    # Edge Detection.
    canny = cv2.Canny(gray, 0, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Blank canvas.
    con = np.zeros_like(img)
    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Keeping only the largest detected contour.
    page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    #con = cv2.drawContours(original_padded, page[:1], -1, (0, 255, 255), 3)


    # Blank canvas.
    con = np.zeros_like(img)
    # Loop over the contours.
    for c in page:
    # Approximate the contour.
        epsilon = 0.02 * cv2.arcLength(c, True)
        corners = cv2.approxPolyDP(c, epsilon, True)
    # If our approximated contour has four points
        if len(corners) == 4:
            break
    cv2.drawContours(original_padded, c, -1, (0, 255, 255), 3)
    cv2.drawContours(original_padded, corners, -1, (0, 255, 0), 10)
    #Sorting the corners and converting them to desired shape.
    corners = sorted(np.concatenate(corners).tolist())

    # Displaying the corners.
    for index, c in enumerate(corners):
        character = chr(65 + index)
        cv2.putText(original_padded, character, tuple(c), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)

    #print(corners)


    pts = np.array(corners)
    print(corners[0])
    print(corners[1])
    print(corners[2])
    print(corners[3])
    #         arrange_points(pts)

    #cv2.drawContours(img, rect_cnts[:1], -1, (0,255,0), 3)

    #fig.add_subplot(rows, columns, i)
    #plt.imshow(original_padded)


    #cv2.imshow('',original_padded)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()


    i += 1


    #     return score
    temp = corners[1]
    corners[1] = corners[2]
    corners[2] = temp

    pts1 = np.float32([[corners[0][0],corners[0][1]],[corners[1][0],corners[1][1]],[corners[2][0],corners[2][1]],[corners[3][0],corners[3][1]]])
    pts2 = np.float32([[0,0],[img.shape[0],0],[0,img.shape[1]],[img.shape[0],img.shape[1]]])

    M = cv2.getPerspectiveTransform(pts1,pts2)

    dst = cv2.warpPerspective(img,M,(img.shape[0],img.shape[1]))
    dst = cv2.resize(dst,(1000,1200))


    cv2.imshow('a',dst)
#     cv2.waitKey()
    cv2.destroyAllWindows()

    smallerArea = dst[int(dst.shape[1]/4):int(dst.shape[1] - dst.shape[1]/20),int(dst.shape[1]/9):int(dst.shape[0] - dst.shape[0]/9)]

#     cv2.imshow('a',smallerArea)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

    mcqs = []
    gray = cv2.cvtColor(smallerArea, cv2.COLOR_BGR2GRAY)
    # gray = cv2.GaussianBlur(gray, (11, 11), 0)
    # Edge Detection.
    canny = cv2.Canny(gray, 50, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 2)))

#     cv2.imshow('a',canny)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    # Blank canvas.
    con = np.zeros_like(smallerArea)
    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    # (contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

    # Plotting the contours
    contoursFrame = smallerArea.copy()
    contoursFrame = cv2.drawContours(contoursFrame, contours, -1, (255, 0, 255), 4)

    # Keeping only the largest detected contour.
    # page = sorted(contours, key=cv2.contourArea, reverse=True)
    #con = cv2.drawContours(original_padded, page[:1], -1, (0, 255, 255), 3)

#     cv2.imshow('a',contoursFrame)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    for i in range(0, len(page)):    #sorting wrt contours
        for j in range(i+1, len(page)):    
            x, y, w, h = cv2.boundingRect(page[i])
            x2, y2, w2, h2 = cv2.boundingRect(page[j])
            if(w*h < w2*h2):    
                temp = page[i];    
                page[i] = page[j];    
                page[j] = temp;           

    count = 0
    xCoords = []
    yCoords = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w > 250 and h >50 and h < 100 and w < 350:# and w < 500:
            mcqs.append(smallerArea[y:y+h,x:x+w])
            xCoords.append(x)
            yCoords.append(y)
    #         cv2.imshow('a',smallerArea[y:y+h,x:x+w])
    #         cv2.waitKey()
    #         cv2.destroyAllWindows()
            count+=1
    print(f'count: {count}')
    if(count < 20):
        return 0

    #Sorting MCQs list with respect to y coordinates first then with respect to x coordinates to make them in order
    for i in range(0, len(mcqs)):    #sorting wrt x
        for j in range(i+1, len(mcqs)):    
            if(xCoords[i] > xCoords[j]):    
                temp = mcqs[i]    
                mcqs[i] = mcqs[j]    
                mcqs[j] = temp   
                temp = xCoords[i]
                xCoords[i] = xCoords[j]
                xCoords[j] = temp
                temp = yCoords[i]
                yCoords[i] = yCoords[j]
                yCoords[j] = temp

    for i in range(0, 10):    #sorting wrt y
        for j in range(i+1, 10):    
            if(yCoords[i] > yCoords[j]):    
                temp = mcqs[i];    
                mcqs[i] = mcqs[j]    
                mcqs[j] = temp 
                temp = xCoords[i]
                xCoords[i] = xCoords[j]
                xCoords[j] = temp
                temp = yCoords[i]
                yCoords[i] = yCoords[j]
                yCoords[j] = temp

    for i in range(10, 20):    #sorting wrt y
        for j in range(i+1, 20):    
            if(yCoords[i] > yCoords[j]):    
                temp = mcqs[i]
                mcqs[i] = mcqs[j] 
                mcqs[j] = temp 
                temp = xCoords[i]
                xCoords[i] = xCoords[j]
                xCoords[j] = temp
                temp = yCoords[i]
                yCoords[i] = yCoords[j]
                yCoords[j] = temp

    #splitting mcqs in 5 regions
    mcqsSplitted=[]
    for i in mcqs:
        j = int((i.shape[1]-40)/5)
        currMcq = []
        currMcq.append(i[:,20:20+j])
        currMcq.append(i[:,j+20:20+j*2])
        currMcq.append(i[:,j*2+20:20+j*3])
        currMcq.append(i[:,j*3+20:20+j*4])
        currMcq.append(i[:,j*4+20:20+j*5])
    #     cv2.imshow('a',currMcq[4])
    #     cv2.waitKey()
    #     cv2.destroyAllWindows()
        mcqsSplitted.append(currMcq)
    #     break

    totalCount=0
    answers = []
    for mcq in mcqsSplitted:
        countOfAnswers=0
        for j in range(0,5):
            hsvb = cv2.cvtColor(mcq[j], cv2.COLOR_BGR2HSV)
            black_lower=np.array([0,0,0],np.uint8)
            black_upper=np.array([255,255,80],np.uint8)
            black=cv2.inRange(hsvb,black_lower,black_upper)

            blackc = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)
            extractCC = largestCC(black)
            ROI = 0
            yellowY, yellowX = np.where(extractCC==0)
            if(len(yellowY) == 0 or len(yellowX) == 0):# no black color
                ROI = 0
            else:#black color
                top, bottom = min(yellowY), max(yellowY)
                left, right = min(yellowX), max(yellowX)
    #             print(top,bottom,left,right)
                ROI = mcq[j][top:bottom, left:right]
                noOfBlackPixels = 0
                for i in range(left,right):
                    for k in range(top,bottom):
                        if extractCC[k,i] == 0:
                            noOfBlackPixels+=1
    #             cv2.imshow('a',extractCC[top:bottom, left:right])
    #             cv2.waitKey()
    #             cv2.destroyAllWindows()
                h = abs(top-bottom)
                w = abs(right-left)
                if noOfBlackPixels > 400:
                    countOfAnswers+=1
                    answers.append(j)
        if(countOfAnswers==0): #no answer
            answers.append(-1)
        elif(countOfAnswers>1): #>1 answers
            for x in range(countOfAnswers):
                answers.pop()
            answers.append(-1)
        totalCount+=1

    #     break
    print(f'totalCount:{totalCount}')

    for i in range(0,len(answers)):
        if answers[i] == 0:
            answers[i] = 'A'
        elif answers[i] == 1:
            answers[i] = 'B'
        elif answers[i] == 2:
            answers[i] = 'C'
        elif answers[i] == 3:
            answers[i] = 'D'
        elif answers[i] == 4:
            answers[i] = 'E'
        else:
            answers[i]='I'

    #SCORING
    print(answers)

    solutionList = []

    # open file and read the content in a list
    with open(r'C:/Users/Umer Mukhtar/Desktop/DIP_Project/sol.txt', 'r') as fp:
        for line in fp:
            # remove linebreak from a current name
            # linebreak is the last character of each line
            x = line[:-1]

            # add current item to the list
            solutionList.append(x)
    
    # open file in write mode
    score = 0
    for i in range(0,len(solutionList)):
        if answers[i] == solutionList[i]:
            score+=1
    print(score)
    finalScore = score
    return totalCount


In [20]:
tC = 0
time.sleep(5)
cap = cv2.VideoCapture('http://192.168.100.11:8080/video')
while(tC!=20):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    tC = evaluateSolutionImage(frame)
    if(finalScore!=0):
        print(f'SCORE:{finalScore}')
        break
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
cap.release()
cv2.destroyAllWindows()


[149, 1452]
[174, 277]
[970, 1461]
[1000, 315]
count: 10
[149, 1453]
[174, 277]
[971, 1462]
[1002, 315]
count: 10
[149, 1454]
[174, 277]
[972, 1463]
[1002, 314]
count: 10
[149, 1456]
[173, 277]
[972, 1464]
[1002, 314]
count: 10
[146, 1465]
[161, 279]
[973, 1470]
[999, 309]
count: 10
[147, 1466]
[162, 277]
[975, 1471]
[1000, 308]
count: 10
[147, 1466]
[163, 276]
[976, 1472]
[1003, 308]
count: 10


KeyboardInterrupt: 

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>